<a href="https://colab.research.google.com/github/arnabd64/Langchain-Guides/blob/main/notebooks/Langchain_Day_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain

In this notebook we will delve into the following components:

1. RunnableLambda
2. RunnableSequence
3. RunnableParallel

* __What is a Runnable?__:

A [Runnable](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface) is a specialized function built by langchain to ease the construction of complex chains. To execute a `Runnable`, there are 3 methods:

1. `.invoke()`: Calls the `Runnable` on an input.
2. `.stream()`: Streams the response of the `Runnable`.
3. `.batch()`: Calls the `Runnable` on a list of inputs.

__Note__: Mostly all the components in langchain are a `Runnable` which includes prompts, LLMs, chains, agents etc.

# Install Libraries

In [1]:
! pip install --no-cache-dir --progress-bar off \
    langchain==0.2.10 \
    langchain_community==0.2.10 \
    python-dotenv \
    > install.log

In [2]:
import os
import dotenv

if not dotenv.load_dotenv("./.env"):
    raise FileNotFoundError

# Load the Components

In [5]:
from langchain_core.runnables import RunnableLambda, RunnableSequence, RunnableBranch, RunnableParallel, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.globals import set_debug
from langchain.utils.math import cosine_similarity
import math

# `RunnableLambda`

A `RunnableLambda` is an interface between a regular method or `lambda` function and a `Runnable` or in simple words, `RunnableLambda` converts a python method into a `Runnable`. Please note that __a `RunnableLambda` method should only take a single input__.

In [10]:
addition = RunnableLambda(lambda x: x + 9)
multiply = RunnableLambda(lambda y: y * 3)

In [13]:
print("Result of Addition:", addition.invoke(6))
print("Result of Multiplication:", multiply.invoke(4))

Result of Addition: 15
Result of Multiplication: 12


# `RunnableSequence`

A `RunnableSequence` is a component that allows __chaining__ of multiple `Runnable` and even other `RunnableSequence`. This component can be considered as the chain in LangChain. You can create a `RunnableSequence` using the `pipe (|)` operator.

__Example__: Let's calculate the Area of a Circle given the radius. The mathematical formula is $A = \pi r^2$. We can break this operation into 2 parts:

1. Compute the square of `radius` ($r^2$)
2. Multiply with `pi` ($\pi$)

We will be using both `RunnableSequence` as well as `pipe` operator.

In [14]:
# RunnableSequence
circle_area = RunnableSequence(
    RunnableLambda(lambda r: r * r, name="Radius-Squared"),
    RunnableLambda(lambda y: y * math.pi, name="Multiply Pi"),
    name="Area of Circle"
)

# pipe operator
circle_area_pipe = (
    RunnableLambda(lambda r: r * r, name="Radius-Squared")
    | RunnableLambda(lambda y: y * math.pi, name="Multiply Pi")
)

set_debug(True)
print("Area of the Circle =", circle_area.invoke(6))

[chain/start] [chain:Area of Circle] Entering Chain run with input:
{
  "input": 6
}
[chain/start] [chain:Area of Circle > chain:Radius-Squared] Entering Chain run with input:
{
  "input": 6
}
[chain/end] [chain:Area of Circle > chain:Radius-Squared] s] Exiting Chain run with output:
{
  "output": 36
}
[chain/start] [chain:Area of Circle > chain:Multiply Pi] Entering Chain run with input:
{
  "input": 36
}
[chain/end] [chain:Area of Circle > chain:Multiply Pi] s] Exiting Chain run with output:
{
  "output": 113.09733552923255
}
[chain/end] [chain:Area of Circle] s] Exiting Chain run with output:
{
  "output": 113.09733552923255
}
Area of the Circle = 113.09733552923255


In [15]:
print("Area of the Circle =", circle_area_pipe.invoke(2))

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": 2
}
[chain/start] [chain:RunnableSequence > chain:Radius-Squared] Entering Chain run with input:
{
  "input": 2
}
[chain/end] [chain:RunnableSequence > chain:Radius-Squared] s] Exiting Chain run with output:
{
  "output": 4
}
[chain/start] [chain:RunnableSequence > chain:Multiply Pi] Entering Chain run with input:
{
  "input": 4
}
[chain/end] [chain:RunnableSequence > chain:Multiply Pi] s] Exiting Chain run with output:
{
  "output": 12.566370614359172
}
[chain/end] [chain:RunnableSequence] s] Exiting Chain run with output:
{
  "output": 12.566370614359172
}
Area of the Circle = 12.566370614359172


# `RunnableParallel`



In [26]:
# description prompt
description_prompt = {
    "template": "You are an assistant tasked with gathering information about a product that an user wants to know about. Get the information for: {product}",
    "input_variables": ["product"]
}

# LLM config
config = {
    "name": "Ollama-Local",
    "base_url": os.getenv("HOST"),
    "model": os.getenv("MODEL"),
    "temperature": float(os.getenv("TEMPERATURE")),
    "timeout": int(os.getenv("TIMEOUT")),
    "keep_alive": 3600
}

# build a chain to gather information
query_chain = (
    PromptTemplate(**description_prompt)
    | Ollama(**config)
    | StrOutputParser()
)

In [27]:
# Pros Chain
#Prompt template
pros_prompt = {
    "input_variables": ["description"],
    "template": "Write down exactly 5 points on why should the user purchase a product according to the following product description:\n{description}"
}

# build the chain
pros_chain = (
    PromptTemplate(**pros_prompt)
    | Ollama(**config)
    | StrOutputParser()
)

In [28]:
# Cons Chain
# Prompt
cons_prompt = {
    "input_variables": ["description"],
    "template": "Write down exactly 5 points on why the user should not purchase a product according to the following product description:\n{description}"
}

# cons chain
cons_chain = (
    PromptTemplate(**cons_prompt)
    | Ollama(**config)
    | StrOutputParser()
)

In [29]:
# Combine the Chains
combine_prompt = {
    "input_variables": ["pros", "cons"],
    "template": "You are tasked with summarizing the advantages and disadvantages of purchasing a product from the given list: Firstly, the advatages:\n{pros} and finally the disadvantages:\n{cons}"
}

# combine chain
combine_chain = (
    PromptTemplate(**combine_prompt)
    | Ollama(**config)
    | StrOutputParser()
)

In [30]:
final_chain = (
    query_chain
    | RunnableParallel(pros=pros_chain, cons=cons_chain)
    | combine_chain
)

In [31]:
set_debug(False)
response = final_chain.invoke({"product": "Porsche 911 Turbo GT"})

In [32]:
print(response)

 Advantages of buying the Porsche 911 Turbo GT include:

1. Brand Reputation: Owning a Porsche 911 Turbo GT associates you with a prestigious luxury sports car brand known for high-performance vehicles, superior engineering, and iconic design.
2. High-Performance Specifications: The estimated 640 horsepower engine provides rapid acceleration (0-60 mph in approximately 2.8 seconds) and an over 205 mph top speed, delivering a thrilling driving experience.
3. Aerodynamic Design: The sleek and aerodynamic body ensures stability at high speeds while maintaining an appealing aesthetic appeal, with improved handling and cornering abilities.
4. Luxurious Interior: The Porsche 911 Turbo GT features a comfortable, ergonomic cabin with sport seats, advanced infotainment systems, and numerous customization options to suit personal preferences.
5. Potential for Exclusivity: Given the non-production status of the car, purchasing one grants access to an exclusive club of early adopters who appreciate

In [33]:
chain_config = {
    # Set the prompts
    "prompts": {
        # prompt to get a description of the product
        "description": {
            "name": "Prompt-Generate-Product_Description",
            "input_variables": ["product"],
            "template": "You are an assistant tasked with gathering information about a product that an user wants to know about. Get the information for: {product}"
        },

        # prompt to get the advantages of a purchase
        "pros": {
            "name": "Prompt-Generate-Advantages",
            "input_variables": ["description"],
            "template": "List exactly 5 reasons why the user should purchase the product described as:\n{description}"
        },

        # prompt to get the disadvantages of a purchse
        "cons": {
            "name": "Prompt-Generate-Disadvantages",
            "input_variables": ["description"],
            "template": "List exactly 5 reasons why the user should not purchase the product described as:\n{description}"
        },

        # prompt to combine both the pros and cons into a single summary
        "combine": {
            "name": "Prompt-Summarize-Outputs",
            "input_variables": ["pros", "cons"],
            "template": "Write down 3 points each on the Advantages and Disadvantages of making a purchase for a product. The advantages of a purcahse are:\n{pros} and the disadvantages of a purcashe are:\n{cons}"
        }
    },

    "llm-config": {
        "name": "Ollama-Local",
        "base_url": os.getenv("HOST"),
        "model": os.getenv("MODEL"),
        "temperature": float(os.getenv("TEMPERATURE")),
        "timeout": int(os.getenv("TIMEOUT")),
        "keep_alive": 3600
    }
}

In [34]:
chain_pipe = (
    # 1. Description Generator Chain
    PromptTemplate(**chain_config["prompts"]["description"])
    | Ollama(**chain_config["llm-config"])
    | StrOutputParser()

    # 2. Parallel chains
    | RunnableParallel(
        # 2A. Generate Advantages
        pros = (
            PromptTemplate(**chain_config["prompts"]["pros"])
            | Ollama(**chain_config["llm-config"])
            | StrOutputParser()
        ),
        # 2B. Generate Disadvantages
        cons = (
            PromptTemplate(**chain_config["prompts"]["cons"])
            | Ollama(**chain_config["llm-config"])
            | StrOutputParser()
        )
    )

    # 3. Combine the Advantages & Disadvantages
    | PromptTemplate(**chain_config["prompts"]["combine"])
    | Ollama(**chain_config["llm-config"])
    | StrOutputParser()
)

In [ ]:
set_debug(False)
response = chain_pipe.invoke({"product": "Porsche 911 Carrera RS"})

In [ ]:
print(response)

In [ ]:
chain_subchain = RunnableSequence(
    # 1. Product Description Generator Sub-Chain
    RunnableSequence(
        PromptTemplate(**chain_config["prompts"]["description"]),
        Ollama(**chain_config["llm-config"]),
        StrOutputParser(),
        name="Chain-Generate-Product_Description"
    ),

    # 2. Generate Advantages & Disadvantages
    RunnableParallel(
        pros = RunnableSequence(
            # 2A. Generate the Advantages
            PromptTemplate(**chain_config["prompts"]["pros"]),
            Ollama(**chain_config["llm-config"]),
            StrOutputParser(),
            name="Chain-Generate-Advantages"
        ),
        cons = RunnableSequence(
            # 2B. Generate the Disadvantages
            PromptTemplate(**chain_config["prompts"]["cons"]),
            Ollama(**chain_config["llm-config"]),
            StrOutputParser(),
            name="Chain-Generate-Disadvantages"
        )
    ),

    # 3. Combine the Results
    RunnableSequence(
        PromptTemplate(**chain_config["prompts"]["combine"]),
        Ollama(**chain_config["llm-config"]),
        StrOutputParser(),
        name="Chain-Summarize-Output"
    ),
    name="Parent-Chain"
)

In [ ]:
set_debug(False)
chain_subchain.invoke({"product": "Porsche Carrera RS"})